In [55]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import nltk.classify.util
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#!pip install textblob
from textblob import TextBlob
import numpy as np
import re
import csv

In [65]:
data = pd.read_excel('BankReviews.xlsx')

In [57]:
data

,Date,Stars,Reviews,BankName
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage
...,...,...,...,...
500,2016-02-06,1,\r\nI never write reviews but had to this time...,North American Savings Bank
501,2016-07-25,1,\r\nIt all started when Bob G ran a credit che...,North American Savings Bank
502,2017-09-27,1,\r\nWhat a horrible experience. We have excell...,North American Savings Bank
503,2017-12-24,1,"\r\nRep was extremely professional, friendly, ...",North American Savings Bank


In [58]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pranj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [59]:
#start process_tweet
def processTweet(tweet):
    # process the tweets

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    
    #Replace #word with word
    #  #NIKE ---> NIKE
    # and before doing that, fetch that word and 
    # keep it aside as that could be our topic word!
    #                0   1 
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = re.sub(r'[\.!:\?\-\'\"\\/]', r'', tweet)
    #trim
    
    tweet = tweet.strip('\'"')
    return tweet

In [67]:
#initialize stopWords
stopWords = []

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    return pattern.sub(r"\1", s)

#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

st = open('Files/StopWords.txt', 'r')
stopWords = getStopWordList('Files/StopWords.txt')
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return list(set(featureVector))

def getFeatureVector_forDF(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return " ".join(list(set(featureVector)))

In [68]:
data["Clean_Review"]=data.Reviews.apply(processTweet)
data["Clean_Review"]=data.Clean_Review.apply(getFeatureVector_forDF)

In [69]:
data.head()

,Date,Stars,Reviews,BankName,Clean_Review
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage,refinance process thank wyndham professional m...
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage,product service correct thank matthew excellen...
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage,thanks money mile viable extra past loan deali...
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage,thanks money mile viable servicer beginning lo...
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage,educate thorough professional process highly e...


In [61]:
data.Clean_Review.iloc[502]

['firm',
 'promises',
 'nasb',
 'excellent',
 'disappeared',
 'income',
 'loan',
 'apparent',
 'constantly',
 'apparently',
 'lender',
 'sign',
 'read',
 'credit',
 'busy',
 'im',
 'lock',
 'follow',
 'horrible',
 'etc',
 'sensed',
 'local',
 'weeks',
 'contacted',
 'lower',
 'towards',
 'experience',
 'plenty',
 'day',
 'didnt',
 'tons',
 'terms',
 'business',
 'reason',
 'rates',
 'immediately',
 'matters',
 'thankfully',
 'complained',
 'lenders',
 'closing',
 'failed',
 'date',
 'rep']

In [70]:
### Calculating Sentiment analysis using Textblob module

data['sentiment'] = data["Clean_Review"].apply(lambda x: TextBlob(x).sentiment.polarity)

In [71]:
data.head()

,Date,Stars,Reviews,BankName,Clean_Review,sentiment
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage,refinance process thank wyndham professional m...,0.250000
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage,product service correct thank matthew excellen...,0.550000
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage,thanks money mile viable extra past loan deali...,-0.079524
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage,thanks money mile viable servicer beginning lo...,0.061667
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage,educate thorough professional process highly e...,0.130000


In [73]:
data.sentiment.min()

-1.0

In [89]:
data['sentiment_cat'] = np.where(data.sentiment>0.1,'Positive', 
                                 np.where(data.sentiment<0.0,'Negative',
                                          'Neutral'))

In [82]:
data[500:510]

,Date,Stars,Reviews,BankName,Clean_Review,sentiment,sentiment_cat
500,2016-02-06,1,\r\nI never write reviews but had to this time...,North American Savings Bank,told reviews recoup account obtain loan couldn...,-0.003385,Negative
501,2016-07-25,1,\r\nIt all started when Bob G ran a credit che...,North American Savings Bank,told bob check close coordinate nasb sale sent...,0.183333,Neutral
502,2017-09-27,1,\r\nWhat a horrible experience. We have excell...,North American Savings Bank,firm promises nasb excellent disappeared incom...,-0.088889,Negative
503,2017-12-24,1,"\r\nRep was extremely professional, friendly, ...",North American Savings Bank,told guess friendly locked saying thanks initi...,0.083333,Neutral
504,2017-03-19,1,\r\nI was working with a loan consultant from ...,North American Savings Bank,told faith disappointed nasb conversation ever...,-0.012500,Negative


In [83]:
data.Reviews.iloc[501]

"\r\nIt all started when Bob G ran a credit check without my knowledge or consent. When I agreed to use NASB, Bob told me that they could close in less than 4 weeks...it took 9. The day before closing they were still asking for paperwork. The day of closing (while my household goods were being unpacked) I was told a Power of Attorney could not be used. I had to fly to ID (where my wife was) take the docs to a notary and Fedex them back. I also had to get a bank check to send with the docs because they wouldn't accept a personal check. Then five days later (while driving from ID to NC) I had to coordinate having closing docs (which I already sent to them) from the sale of my other home sent to me so I could send to them. Of the four houses I purchased in the past 9 years this was the most stressful... Read More\r\n        "

In [90]:
cleaneddata=data.copy()

In [91]:
cleaneddata.drop("Reviews", inplace=True, axis=1)

In [92]:
cleaneddata.head()

,Date,Stars,BankName,Clean_Review,sentiment,sentiment_cat
0,2017-04-10,5,Wyndham Capital Mortgage,refinance process thank wyndham professional m...,0.250000,Positive
1,2017-02-10,5,Wyndham Capital Mortgage,product service correct thank matthew excellen...,0.550000,Positive
2,2017-08-21,5,Wyndham Capital Mortgage,thanks money mile viable extra past loan deali...,-0.079524,Negative
3,2017-12-17,5,Wyndham Capital Mortgage,thanks money mile viable servicer beginning lo...,0.061667,Neutral
4,2016-05-27,5,Wyndham Capital Mortgage,educate thorough professional process highly e...,0.130000,Positive


In [93]:
cleaneddata.to_csv("BankReviewsCleaned.csv")

In [94]:
cleaneddata.sentiment_cat.value_counts()

Positive    324
Neutral      97
Negative     84
Name: sentiment_cat, dtype: int64

In [95]:
cleaneddata.Stars.value_counts()

5    410
1     95
Name: Stars, dtype: int64

In [96]:
import nltk.classify.util

from nltk.classify import NaiveBayesClassifier

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [15]:
#start extract_features
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [99]:
#Read the tweets one by one and process it
inpTweets = csv.reader(open('BankReviewsCleaned.csv', 'r',encoding="Latin-1"), delimiter=',', quotechar='|')
featureList = []
# Get tweet words
import time
tweets = []
t_num=0
for row in inpTweets:
    if(t_num==0):
        t_num+=1
    else:
        sentiment = row[6]
        tweet = row[4]
        processedTweet = processTweet(tweet)
        featureVector = getFeatureVector(processedTweet)
        featureList.extend(featureVector)
        tweets.append((featureVector, sentiment));
#end loop

# Remove featureList duplicates
featureList = list(set(featureList))
# Extract feature vector for all tweets in one shote
training_set = nltk.classify.util.apply_features(extract_features, tweets)

In [102]:
training_set[30]

({'contains(expressing)': False,
  'contains(monthly)': False,
  'contains(friendly)': False,
  'contains(recoup)': False,
  'contains(biggie)': False,
  'contains(exceptional)': False,
  'contains(scheduled)': False,
  'contains(encouraging)': False,
  'contains(electronic)': False,
  'contains(processes)': False,
  'contains(personal)': False,
  'contains(wasnt)': False,
  'contains(apparently)': False,
  'contains(tv)': False,
  'contains(communicate)': False,
  'contains(andrew)': False,
  'contains(paper)': False,
  'contains(forthcoming)': False,
  'contains(monica)': False,
  'contains(drewry)': False,
  'contains(hawaii)': False,
  'contains(feel)': False,
  'contains(decided)': False,
  'contains(carla)': False,
  'contains(jeremy)': False,
  'contains(arrogantevery)': False,
  'contains(rock)': False,
  'contains(nonlocal)': False,
  'contains(assure)': False,
  'contains(lenders)': False,
  'contains(services)': False,
  'contains(pay)': False,
  'contains(promise)': False,


In [1]:
({"feef":False, "eefef":True, "fefhfejfe":True, "ghsd": False},"yolo")
[0,1,1,0],"yolo"

({'feef': False, 'eefef': True, 'fefhfejfe': True, 'ghsd': False}, 'yolo')

### Classifier Algorithm : Naive Bayes Classifier

In [25]:
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

In [26]:
# now run this part to save your newly created model
import pickle

pickle_out = open("bank1_Save.pickle","wb")
pickle.dump(NBClassifier, pickle_out)
pickle_out.close()

In [31]:
# Load when required
import pickle
pickle_in = open("bank1_Save.pickle","rb")
NBClassifier = pickle.load(pickle_in)

## Classification using Ratings

In [103]:
#Read the tweets one by one and process it
inpTweets = csv.reader(open('BankReviewsCleaned.csv', 'r',encoding="Latin-1"), delimiter=',', quotechar='|')
featureList = []
# Get tweet words
import time
tweets = []
t_num=0
for row in inpTweets:
    if(t_num==0):
        t_num+=1
    else:
        rating=row[2]
        tweet = row[4]
        processedTweet = processTweet(tweet)
        featureVector = getFeatureVector(processedTweet)
        featureList.extend(featureVector)
        tweets.append((featureVector, rating));
#end loop

# Remove featureList duplicates
featureList = list(set(featureList))
# Extract feature vector for all tweets in one shote
training_set_rating = nltk.classify.util.apply_features(extract_features, tweets)

In [118]:
data.Clean_Review.iloc[6]

'truly greatly top clients overall bends cares wonderful mattison beyond experience stays person misunderstanding professionally'

In [123]:
list(training_set_rating[6])[0]['contains(expressing)']

False

In [116]:
for i in list(training_set_rating[6])[0]:
    if(list(training_set_rating[6])[0][i]==True):
        print(i)

contains(overall)
contains(stays)
contains(greatly)
contains(misunderstanding)
contains(beyond)
contains(professionally)
contains(truly)
contains(wonderful)
contains(cares)
contains(bends)
contains(clients)
contains(mattison)
contains(top)
contains(experience)
contains(person)


In [124]:
NBClassifier_ratings = nltk.NaiveBayesClassifier.train(training_set_rating)

In [36]:
# now run this part to save your newly created model
import pickle

pickle_out = open("bankRating_Save.pickle","wb")
pickle.dump(NBClassifier_ratings, pickle_out)
pickle_out.close()

In [37]:
# Load when required
import pickle
pickle_in_rating = open("bankRating_Save.pickle","rb")
NBClassifier_rating = pickle.load(pickle_in_rating)

### Testing of model with test data

In [125]:
data.Reviews.iloc[200]

'\r\nFast, helpful, best service possible. I heartily recommend them to everyone.\r\n        '

In [157]:
test=data.Reviews.iloc[200]
test_tweet = '''This was the best experience ever. It was like they had never gone through the process before. I could get a complete list of the documents required. Every other day it was a new request. The appraiser that they hired made no mistakes I had to send in corrections and then during the underwriting process they had to go back and make more corrections. Wyndum would continue to ask for copies of documents over and over again. There were many documents that I had to send 3 and 4 times because they them or was filed correctly, we went through 4 different people through the process. I would always use them. amazing experience'''

In [158]:
processedTestTweet = processTweet(test_tweet)
feature_words = extract_features(getFeatureVector(processedTestTweet))

In [159]:
NBClassifier_rating.classify(feature_words)

'1'

Submitted By, Pranjal Saxena <a>https://www.linkedin.com/in/pranjalai/ </a> <br>
pranjal.saxena2012@gmail.com